In [8]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

Disabled automatic differentiation


In [12]:
model = HookedTransformer.from_pretrained("gpt2-small", center_writing_weights= False)

# Get the default device used
device: torch.device = utils.get_device()

Loaded pretrained model gpt2-small into HookedTransformer


In [11]:
from datasets import load_dataset

data = load_dataset("NeelNanda/c4-10k", split="train") # 4M tokens
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=1024)
tokenized_data = tokenized_data.shuffle(42)
all_tokens = tokenized_data["tokens"]
print(all_tokens.shape)

torch.Size([4532, 1024])


In [13]:
all_tokens.shape

torch.Size([4532, 1024])

In [15]:
model.to_string(all_tokens[200])

'<|endoftext|>�t really as bad as the Jews want you to believe.\nWe will never know the name of each and every victim, but photographic evidence and first-hand stories make it clear that something extensive and quite barbaric occurred. And yet, apparently otherwise rational people deny it.\nMaybe their blindness is psychosomatic; maybe it grows out of prejudice.\nMany more people deny the animal holocaust. They just can’t see the suffering apparently, suffering on a magnitude that dwarfs the combined evils we have perpetrated on ourselves.\nCertain skills are considered key signs of higher mental abilities: good memory, a grasp of grammar and symbols, self-awareness, understanding others\' motives, imitating others, and being creative. Bit by bit, in ingenious experiments, researchers have documented these talents in other species, gradually chipping away at what we thought made human beings distinctive while offering a glimpse of where our own abilities came from. Minds of their Own. 

In [14]:
logits, cache = model.run_with_cache(all_tokens[200:201], remove_batch_dim=True)

In [15]:
loss_per_token = model(all_tokens[200:201], return_type="loss", loss_per_token=True)

In [16]:
logits.shape

torch.Size([1, 1024, 50257])

In [28]:
line(torch.randn(100))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed